In [ ]:
import pandas as pd
import numpy as np
import os
import boto3
import datetime

from config import ACCESS_KEY, SECRET_KEY, TOKEN

TIME_ZONE = os.getenv("TIME_ZONE", "Europe/Moscow") #настройка функции


pd.set_option('display.max_columns',None)
pd.set_option('display.widfh',None)
pd.set_option('display.max_colwidfh',None)


In [2]:
def get_now_datetime_str(): # получаем актуальное время
    time_zone = os.getenv("TIME_ZONE", "Europe/Moscow") # меняем таймзону на московскую
    now = datetime.datetime.now(pytz.timezone(time_zone))    
    yesterday = now - datetime.timedelta(days=1) #нужна вчерашняя дата так как данные за прошлый день
    last_month_data = now - relativedelta(month=1)
    return {'key_parquet': yesterday.strftime('year=%Y/month=%m/%d.parquet'),
            'key': yesterday.strftime('year=%Y/month=%m/%d.csv'),
            'key_month': yesterday.strftime('year=%Y/month=%m.csv'),
            'now':now.strftime('%Y-%m-%d %H:%M:%S'),
            'yesterday_data':yesterday.strftime('%Y-%m-%d'),
            'yesterday':yesterday.strftime('%Y-%m-%d %H:%M:%S'), 
            'year':yesterday.strftime('%Y'),
            'month':yesterday.strftime('%m'),
            'day':yesterday.strftime('%d'),
            'last_month_data':last_month_data.strftime('%Y-%m-%d')
            }

def get_s3_instance(): # функция создает соединение
    session = boto3.session.Session()
    return session.client(
        aws_access_key_id=ACCESS_KEY,
        aws_secret_access_key=SECRET_KEY,
        service_name='s3',
        endpoint_url='https://storage.yandexcloud.net'
    )

def upload_dump_to_s3(): # функция выгружает данные в s3
    get_s3_instance().upload_file(
        Filename=TEMP_FILENAME,
        Bucket=BUCKET_NAME,
        Key=key
    )

def remove_temp_files(): #функция удаляет временный файл
    os.remove(TEMP_FILENAME)

s3 = get_s3_instance()

In [ ]:
file_path = '/home/boris/Desktop/sms/sms_messages_02.06.2025.csv'
df = pd.read_csv(file_path, encoding= 'cp1251', sep=';')
df = df.drop(['Дата.1'],axis=1)
df['Цена'] = df['Цена'].apply(lambda x: float(x.replace(',','.')))
df['Дата'] = pd.to_datetime(df['Дата'], format='mixed')
df = df[df['Дата'] < datetime.datetime.strptime('2025-06-01', '%Y-%m-%d')]

In [ ]:
last_date = df['Дата'].max().date().strftime('%Y-%m-%d')
last_date

'2025-05-31'

In [ ]:
year = df['Дата'].max().date().strftime('%Y')
month = df['Дата'].max().date().strftime('%m')

In [ ]:
month

'05'

In [73]:
file_path = '/home/boris/Desktop/sms/sms_messages_02.06.2025.csv'
df = pd.read_csv(file_path, encoding= 'cp1251', sep=';')
df = df.drop(['Дата.1'],axis=1)
df['Цена'] = df['Цена'].apply(lambda x: float(x.replace(',','.')))
df['Дата'] = pd.to_datetime(df['Дата'], format='mixed')
df = df[df['Дата'] < datetime.datetime.strptime('2025-06-01', '%Y-%m-%d')]

last_date = df['Дата'].max().date().strftime('%Y-%m-%d')
year = df['Дата'].max().date().strftime('%Y')
month = df['Дата'].max().date().strftime('%m')
TEMP_FILENAME = f'expense_smsc_{year}_{month}.csv'
df.to_csv(TEMP_FILENAME, sep=';', index=False)

folder = 'expense_smsc'
BUCKET_NAME = 'dwh-asgard'
key = f'{folder}/year={year}/month={month}/{month}.csv'

upload_dump_to_s3()
remove_temp_files()
